#BMA-based GDA scoring

This code is used for BMA-based score calculation. Each gene-disease pair is read, their relevant phenotypic embeddings are extracted, and cosine similarity between MP and HP is calculated. BMA is then applied as a reflection for a particular gene-disease association.

Dependencies:
numpy==1.26.4
scipy==1.13.1

In [ ]:
#imports needed

import numpy as np
from scipy.spatial.distance import cosine
from tqdm import tqdm
from itertools import product
from multiprocessing import Pool, cpu_count

In [ ]:
#calculates BMA in two-way approach
def calculate_best_match_average(mp_id, hp_id, mp_annotations, hp_annotations, embeddings):
    mp_phenotypes = [p for p in mp_annotations.get(mp_id, []) if p in embeddings]
    hp_phenotypes = [p for p in hp_annotations.get(hp_id, []) if p in embeddings]

    if not mp_phenotypes or not hp_phenotypes:
        return 0.0

    # MP → HP
    mp_to_hp_scores = []
    for mp in mp_phenotypes:
        mp_vec = embeddings[mp]
        max_sim = max(1 - cosine(mp_vec, embeddings[hp]) for hp in hp_phenotypes)
        mp_to_hp_scores.append(max_sim)
    avg_mp_to_hp = sum(mp_to_hp_scores) / len(mp_to_hp_scores)

    # HP → MP
    hp_to_mp_scores = []
    for hp in hp_phenotypes:
        hp_vec = embeddings[hp]
        max_sim = max(1 - cosine(hp_vec, embeddings[mp]) for mp in mp_phenotypes)
        hp_to_mp_scores.append(max_sim)
    avg_hp_to_mp = sum(hp_to_mp_scores) / len(hp_to_mp_scores)

    return (avg_mp_to_hp + avg_hp_to_mp) / 2

#to load embeddings
def load_embeddings(filepath):
    embeddings = {}
    with open(filepath, 'r') as file:
        for line in file:
            if line.strip() and not line.startswith("Entity Embeddings:") and line.startswith("http://purl.obolibrary.org/obo/"):
                parts = line.strip().split()
                if len(parts) >= 2:
                    entity_id = parts[0].split('/')[-1]
                    try:
                        vector = np.array([float(x) for x in parts[1:]])
                        embeddings[entity_id] = vector
                    except ValueError:
                        continue
    return embeddings

#read and extract IDs from phenotype files
def read_phenotype_file(filepath):
    phenotypes = {}
    with open(filepath, 'r') as file:
        for line in file:
            if line.strip():
                parts = line.strip().split('\t')
                if len(parts) == 2:
                    entity_id = parts[0].split('/')[-1]
                    phenotype_id = parts[1].split('/')[-1]
                    if entity_id not in phenotypes:
                        phenotypes[entity_id] = []
                    phenotypes[entity_id].append(phenotype_id)
    return phenotypes

if __name__ == "__main__":
    # File paths
    embedding_file_path = '/content/drive/My Drive/colab_output/embedding_file_example.txt'
    gene_phenotypes_file = '/content/drive/My Drive/g_p.txt'
    disease_phenotypes_file = '/content/drive/My Drive/d_p.txt'
    gene_diseases_file = '/content/drive/My Drive/g_d.txt'

    #embeddings and phenotype annotations loaded
    embeddings = load_embeddings(embedding_file_path)
    mp_annotations = read_phenotype_file(gene_phenotypes_file)
    hp_annotations = read_phenotype_file(disease_phenotypes_file)

    #reading gene-disease pairs
    gene_diseases = []
    with open(gene_diseases_file, 'r') as file:
        for line in file:
            if line.strip():
                parts = line.strip().split('\t')
                if len(parts) == 2:
                    mgi_id = parts[0].split('/')[-1]
                    omim_id = parts[1].split('/')[-1]
                    gene_diseases.append((mgi_id, omim_id))

    #saving results o/p
    output_best_match_bma_filepath = '/content/drive/My Drive/colab_output/scores_file_example.txt'

    #processing pairs
    def process_pair(pair):
        mgi_id, omim_id = pair
        return (f"MGI:{mgi_id} ↔ OMIM:{omim_id}", calculate_best_match_average(mgi_id, omim_id, mp_annotations, hp_annotations, embeddings))


    all_pairs = list(product(set([pair[0] for pair in gene_diseases]), set([pair[1] for pair in gene_diseases])))

    #parallelize processing
    with Pool(processes=cpu_count()) as pool:
        best_match_bma_results = list(tqdm(pool.imap(process_pair, all_pairs), total=len(all_pairs), desc="Calculating best match average BMA"))

    #descending order of score sorting
    best_match_bma_results.sort(key=lambda x: x[1], reverse=True)

    #saving
    with open(output_best_match_bma_filepath, 'w') as file:
        file.write("Gene-Disease Association\tBMA Similarity\n")
        for gene_disease_pair, bma_similarity in best_match_bma_results:
            file.write(f"{gene_disease_pair}\t{bma_similarity:.4f}\n")

    print(f"Best match average BMA similarities for gene-disease associations saved to {output_best_match_bma_filepath}.")